In [ ]:
import shutil
from subprocess import check_output
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import os
#import matplotlib.pyplot as plt
import pandas as pd
#from shapely.geometry import Polygon,
import RClib as RClib
#import objectdetectionlib as ODlib



In [ ]:
%load_ext autoreload
%autoreload 2

config = RClib.loadconfigs('.\config_sedimentcores.json')

if Path(config['workspace'] + '/rcprocessingdf.pkl').is_file() and config['use_initialdf'] :
    print('Read scandf from pickle')
    scandf = pd.read_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))
else:
    scandf = RClib.provide_scandf(inputdirectory=config['workspace'], imageformat = config['baseimageformat'])
    if config['use_startpointtransformation']:
        scandf = scandf.apply(RClib.createGCPfile_forsedimentcores, axis=1)
    scandf= scandf.apply(RClib.defineRawTherapeeOutput, foldername='02_TIF', axis=1)
    scandf= scandf.apply(RClib.defineRealityCaptureOutput, foldername='03_RC', axis=1)
    #scandf.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))

for index, scan in scandf.iterrows():
    #RClib.visualize_rcbox(scan['rcbox'])
    print('Processing scan: ' + scan['id'])
    print('Number of images: ' + str(len(scan['imagedf'])))
    #print('Number of masks: ' + str(len(scan['imagedf'][scan['imagedf']['maskimg_path'].notnull()])))


In [ ]:
scandf10 = pd.DataFrame()
if config['texture']:
    for index,scan in scandf9.iterrows():
        scan = RClib.readProcessingstateFile(scan)
        if RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'):
            if not RClib.checkProcessingstate(scan,command='commandlist_texturingsettings'):
                scan['commandlist_texturingsettings']=Path(config['texturingsettings'])
                print('Texturing settings set for ', scan['id'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_texturingsettings')
            if not RClib.checkProcessingstate(scan,command='texturing-rccmdpath'):
                scan['commandlist_texturing'] = []
                if 'sedimentcores_selectstraight' in config and config['sedimentcores_selectstraight']:
                    scan['commandlist_texturing'].append('-selectComponent ' + str(scan['id']) + '_frontal')
                scan['commandlist_texturing'].append('-selectModel ' + scan['id'] + '_highpoly')
                #scan['commandlist_texturing'].append('-selectLargestModelComponent')
                #scan['commandlist_texturing'].append('-invertTrianglesSelection')
                scan['commandlist_texturing'].append('-smooth')
                scan['commandlist_texturing'].append('-correctColors geometry') 
                scan['commandlist_texturing'].append('-unwrap') 
                scan['commandlist_texturing'].append('-calculateTexture')
                scan['commandlist_texturing'].append('-simplify '+ str(config['simplify_settings']))
                scan['commandlist_texturing'].append('-renameSelectedModel ' + scan['id'] + '_100k')
                scan['modelspaths'] = {'modelpath_100k': scan['Resultoutputfolder'] / Path(config['exportname_prefix'] + scan['id'] + '_100k.obj')}
                print(scan['modelspaths'])
                scan['commandlist_texturing'].append('-exportModel ' + scan['id'] + '_100k ' + str(scan['modelspaths'].get('modelpath_100k')) + ' ' + config['exportsettings1'] )
                scan['commandlist_texturing'].append('-save')
                scan['commandlist_texturing'].append('-quit')  
                scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_texturing', rccmdpathfield='texturing-rccmdpath')
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='texturing-rccmdpath' )
                RClib.writeProcessingstateFile(scan)
        scandf10 = scandf10.append(scan)
else:
    scandf10=scandf9


In [ ]:
if config['orthomosaic']:
    for index,scan in scandf3.iterrows():
        scan = RClib.readProcessingstateFile(scan)
        if RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'):
            if not RClib.checkProcessingstate(scan,command='commandlist_orthoimage'):
                if 'constantorthobox' in config.keys() and Path(config['constantorthobox']).is_file():
                    scan['orthoboxpath'] = Path(config['constantorthobox'])
                    scan['commandlist_orthoimage'] = []
                    scan['commandlist_orthoimage'].append('-selectModel ' + scan['id'] + '_highpoly') 
                    scan['commandlist_orthoimage'].append('-calculateOrthoProjection ' + scan['orthoboxpath'] ) 
                    scan['commandlist_orthoimage'].append('-exportOrthoProjection ' + scan['orthoboxpath'] ) 
            

In [ ]:
%load_ext autoreload
%autoreload 2
if config['orthobox2gis']:
    for index,scan in scandf.iterrows():
        #print(scan.keys())
        RClib.read_rcorthobox(scan)
        #print(scan['orthobox'])
        scan['orthobox'].to_file(config['workspace'] + '/Orthoboxes_utm38.shp', driver='ESRI Shapefile')